# Predictions for validation period
21st February 2024

In [2]:
import sys
sys.path.append('/Users/kenzatazi/Documents/CDT/Code/bcm4rcm')

In [3]:
import numpy as np
import xarray as xr
import glob
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cf
import matplotlib.ticker as mticker
import utils.areal_plots as aplot
import utils.prob_plots as pplot

In [4]:
from models import moe

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#set plot font size
plt.rcParams.update({'font.size': 16})

## Calculate weights

In [5]:
# pass the wassertein distance to softmax function

wass = np.load('/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/weights/wass_arr.npy')

In [6]:
wass.shape

(5, 12, 90, 40)

In [7]:
T = 8  # temperature

In [8]:
weights = moe.softmax(wass, T)

In [9]:
weights = weights.reshape(5, 12, 90, 40)

## Predictions

In [10]:
rcm_list = []

for file in glob.glob('/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/processed/historical*1975.csv'):
    print(file)
    df = pd.read_csv(file, index_col=0)
    loc_df =  df

    loc_df['time'] = pd.to_datetime(loc_df['time'])
    loc_df['month'] = loc_df['time'].dt.month
    loc_df = loc_df.sort_values(by=['month', 'lon', 'lat'])
    
    #p95 = np.percentile(loc_df['tp'], 95)
    loc_df['tp_scale'] = loc_df['tp']
    
    loc_df_gr = loc_df.groupby(['month', 'lon', 'lat']).mean().reset_index()

    rcm_list.append(loc_df_gr[['month', 'lon', 'lat', 'tp_scale']].values.reshape(12, 90, 40, 4))

/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/processed/historical_MPI_IITM_1961_1975.csv
/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/processed/historical_CSIRO_1961_1975.csv
/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/processed/historical_MPI-REMO2009_1961_1975.csv
/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/processed/historical_NOAA_SMHI_1961_1975.csv
/Users/kenzatazi/Documents/CDT/Code/bcm4rcm/data/processed/historical_MIROC_1961_1975.csv


In [11]:
rcm_stack = np.stack(rcm_list, axis=0)

In [12]:
mix_exp = moe.mixture_of_experts(weights, rcm_stack[:,:,:,:, 3])

In [13]:
np.save('validation_mix_exp_T8.npy', mix_exp)

##